# Magnetic Dipole App

Now that we have learned how to interact with `geoh5`, we can start implementing our first application. We are going to create a magnetic dipole simulator from scratch, which could become a useful tool to interpret magnetic data maps.


![mag_dipole_gif](./images/mag_dipole_app.gif "segment")

## Background

Python needs to import external libraries for certain classes and functions to be recognized when running the code. That is done below.

In [1]:
# Import the necessary packages
from __future__ import annotations

import sys
import numpy as np

from geoh5py.data import Data
from geoh5py.objects import ObjectBase, Points
from geoh5py.ui_json import InputFile, constants, templates
from geoh5py.ui_json.utils import monitored_directory_copy
from geoh5py.workspace import Workspace

### Arrays
We will make use a few useful methods on `numpy` arrays.
- The `sum()` method applied to a specific axis of an array, collapsing the dimension.
- The `dot()` method performs the dot product between two arrays
- The transpose (`.T`) method to get the dimensions of arrays to align:

$[1 \times 3] \cdot [M \times 3].T \rightarrow [1 \times M] $.

- Array broadcasting (repeat) of a 1D array along a second dimension using the `[:, None]` indexing:

$[N \times 3] \;/\;([1 \times N][: None]) \rightarrow [N \times 3] $

## The physics

The strength of the magnetic field flux of a single dipole is:

$$\mathbf{B} = \frac{\mu_0}{4 \pi} \left( \frac{3 \mathbf{r} (\mathbf{m} \cdot \mathbf{r})}{r^5} - \frac{\mathbf{m}}{r^3} \right)$$

where $\mu_0$ is a constant ($4 \pi \times 10^{-7}$), $\mathbf{m}$ is the magnetic dipole moment (susceptibility $\times$ volume), and $\mathbf{r}$ is the vector between the dipole and the locations.

In a geophysical context however, we normally get Total Magnetic Intensity (TMI) data from magnetometers. The conversion can be approximated by a projection of the fields onto the direction of Earth's inducing field.

$$
b_{TMI} \approx \mathbf{\hat H}_0 \cdot \mathbf{b}
$$

So we will need at least to
 - Create functions for:
    - Computing the fields of a dipole(s)
    - Converting vector fields to TMI.
 - Provide inputs for parameters:
    - Receivers (observation) locations
    - Source dipole locations
    - Dipole moment values
    - Dipole inclination angles
    - Dipole declination angles
    - Earth's field inclination
    - Earth's field declination

## Functions

### Dipole function

We can begin with writing the function needed to compute the
magnetic field from a single dipole. The function should be able to take as argument

    - Coordinates of the source and receivers (defining 'r'),
    - Parameters defining the strength (moment) and orientation (inc, dec) of the dipole source.

In [11]:
def b_field(source, locations, moment, inclination, declination):
    """
    Compute the magnetic field components of a dipole on an array of locations.

    :param source: Location of a point dipole, shape(1, 3).
    :param locations: Array of observation locations, shape(n, 3).
    :param moment: Dipole moment of the source (A.m^2)
    :param inclination: Dipole horizontal angle, clockwise from North
    :param declination: Dipole vertical angle from horizontal, positive down

    :return: Array of magnetic field components, shape(n, 3)
    """
    # Convert the inclination and declination to Cartesian vector
    m = moment * inclination_declination_2_xyz(inclination, declination)

    # Compute the radial components
    rad = source - locations

    # Compute |r|
    dist = np.sum(rad**2.0, axis=1) ** 0.5

    # mu_0 / 4 pi  * 1e9 for nT
    constant = 100
    fields = constant * (
        ((np.dot(m, rad.T).T * 3 * rad) / dist[:, None] ** 5 ) -
        (m / dist[:, None] ** 3))

    return fields

### Inclination to declination conversion
We require a function to convert inclination/declination angles to Cartesian vectors. The same function will come handy later when computing the TMI projection.

In [10]:
def inclination_declination_2_xyz(inclination, declination):
    """Convert inclination and declination angles (degrees) to unit vector (xyz)."""
    theta = np.deg2rad((90 - declination) % 360)
    phi = np.deg2rad(90 + inclination)
    xyz = np.c_[np.sin(phi) * np.cos(theta), np.sin(phi) * np.sin(theta), np.cos(phi)]

    return xyz

We can now test our `b_field` function with existing objects present in our `geoh5` project created and saved at the beginning of the workshop.

In [8]:
with Workspace("C:\\your_file_here.geoh5") as workspace:
    # get variables
    source = workspace.get_entity("SunCityLocation")[0].vertices
    output_entity = workspace.get_entity("SunCity")[0]
    locations = output_entity.centroids
    m = 50**3 * 0.05 * 57000
    
    # compute fields
    field = b_field(source, locations, m, -67, -17)
    
    # write back to workspace
    output_entity.add_data(
        {
            "bx": {"values": field[:, 0]},
            "by": {"values": field[:, 1]},
            "bz": {"values": field[:, 2]},
        }
    )

AttributeError: 'NoneType' object has no attribute 'vertices'

The `b_field` function returns an array for the three components of the
magnetic field due to the dipole source.

Let's write a simple function to get the TMI (projected) data instead.

In [12]:
def tmi_projection(b_components, earth_field):
    """Project magnetic field onto Earth's field."""
    h0 = inclination_declination_2_xyz(earth_field[0], earth_field[1])
    return np.dot(h0, b_components.T)

and test it for the inducing field parameters at the test site (single location):

In [17]:
with Workspace("../assets/suncity.geoh5") as workspace:
    # get variables
    source = workspace.get_entity("SunCityLocation")[0].vertices
    output_entity = workspace.get_entity("SunCity")[0]
    locations = output_entity.centroids
    m = 50**3 * 0.05 * 57000
    
    # compute fields
    earth_field = np.array([-45,0])
    field = b_field(source, locations, m, earth_field[0], earth_field[1])
    tmi = tmi_projection(field, earth_field)
    
    # write back to workspace
    output_entity.add_data({"tmi_45": {"values": tmi}})

## The Simulator (creating a generic function)

In this section, we are going to string together the functions defined above and generalize our approach so that we can deal with different types of `geoh5` objects. Let's wrap all the previous functions into a `magnetic_simulator` class.

In [5]:
def magnetic_simulator(
    sources: ObjectBase,
    receivers: ObjectBase,
    moments: Data | float,
    inclinations: Data | float,
    declinations: Data | float,
    earth_inc: float,
    earth_dec: float,
):
    def locations(entity):
        if hasattr(entity, "centroids"):
            return entity.centroids

        return entity.vertices

    # Extract dipole coordinates
    dipoles = locations(sources)

    # Extract receiver coordinates
    observations = locations(receivers)

    def vectorize(entity):
        if isinstance(entity, Data):
            return entity.values

        return np.ones(dipoles.shape[0]) * entity

    # Get dipole moment values
    mom = vectorize(moments)
    # Get dipole inclination values
    inc = vectorize(inclinations)
    # Get dipole declination values
    dec = vectorize(declinations)

    # Loop over the dipoles
    fields = np.zeros_like(observations)

    for source, moment, inclination, declination in zip(dipoles, mom, inc, dec):
        fields += b_field(source, observations, moment, inclination, declination)

    tmi = tmi_projection(fields, (earth_inc, earth_dec))

    # Add data to receiver object
    data = receivers.add_data(
        {
            "b_x": {"values": fields[:, 0]},
            "b_y": {"values": fields[:, 1]},
            "b_z": {"values": fields[:, 2]},
            "tmi": {"values": tmi}
        }
    )

    return data

We now have a generic function to compute TMI data based on almost any type of ANALYST object.

Let us edit our Points object to add more vertices and create data arrays for moment, azimuth and dips. We can now run the simulation by simply giving those entities to our class.

In [3]:
with Workspace("C:\\your_file_here.geoh5") as workspace:
    # get variables
    source = workspace.get_entity("SunCityLocation")[0]
    output_entity = workspace.get_entity("SunCity")[0]
    
    m = 50**3 * 0.05 * 57000
    
    #
    data = magnetic_simulator(source, output_entity, m, -67,-17,-67,-17)
    print(data)

NameError: name 'Workspace' is not defined

![bfield](./images/b_field.png)

## The Driver (Running with Geoscience ANALYST)

In the previous section, we created our simulation program called `magnetic_simulator` that could compute the fields of dipoles defined by `geoh5` object. As a final step, we need to define a `driver` program that can be called from ANALYST.

We are going to take advantage of the
[geoh5py.ui_json.InputFile](https://geoh5py.readthedocs.io/en/stable/content/api/geoh5py.ui_json.html#geoh5py.ui_json.input_file.InputFile)
class to handle value conversions between geoh5 and python, as well as the `monitored_directory_copy` function to handle the live link with ANALYST.

In [6]:
def run(file: str):
    
    uifile = InputFile.read_ui_json(file).data
    
    with uifile["geoh5"].open(mode="r+"):
        magnetic_simulator(
            uifile["dipoles"],
            uifile["locations"],
            uifile["moments"],
            uifile["inclinations"],
            uifile["declinations"],
            uifile["earth_inc"],
            uifile["earth_dec"])
        
        if uifile["monitoring_directory"] is not None:
            monitored_directory_copy(uifile["monitoring_directory"], uifile["receivers"])

The final step requires to make this program available to python to be executed from ANALYST. That requires going from a notebook to a .py file. This is done by:

- Use the `File\Download as\Python *.py` dropdown menu to convert this notebook to a `py` file.
- Rename the file to `mag_dipole_app.py`.
- Remove everything in the file except the
    - the imports,
    - the blocks of functions `def`
    - the following lines (without #'s and $\textbf{no}$ indentation)

    ```
    if __name__ == "__main__":
        file = sys.argv[1]
        run(file)
    ```

This part becomes the entry point of the python interpreter when running the
`mag_dipole_app.py` script.

## The User-Interface (ui.json)

In the previous section covering the [geoh5 API](geoh5-API), we have introduced functionality to compute the magnetic field from dipole locations. We are now going to create a user-interface (UI) to be able to run this program from a Geoscience ANALYST session.

We now can create our ui.json to provide inputs for the following 7 parameters:

- `Object` defining the dipoles (sources)
- `Object` defining the receivers
- `Data` or `float` value for the dipole moments
- `Data` or `float` value for the dipole inclination angles
- `Data` or `float` value for the dipole declination angles
- `Float` value for Earth's field inclination angle
- `Float` value for Earth's field declination angle

Let's create a UI.json with forms for each one of those inputs

In [14]:
import json
ui_file = constants.default_ui_json.copy()
ui_file["title"] = "Magnetic Dipole Application"
ui_file["conda_environment"] = "python-training"
ui_file["run_command"] = "magnetic_dipole_app"
ui_file.update({"dipoles": templates.object_parameter(label="Dipoles", mesh_type=Points.default_type_uid)})
ui_file.update({"locations": templates.object_parameter(label="locations")})
ui_file.update({"moments": templates.data_value_parameter(label="Moments", association="Vertex",parent="dipoles", value=1.0)})
ui_file.update({"inclinations": templates.data_value_parameter(label="Dipole inclinations", association="Vertex",parent="dipoles", value=-67.0)})
ui_file.update({"declinations": templates.data_value_parameter(label="Dipole declinations", association="Vertex",parent="dipoles", value=-17.0)})
ui_file.update({"earth_inc": templates.object_parameter(label="Earth inclination", value=-67.0)})
ui_file.update({"earth_dec": templates.object_parameter(label="Earth declination", value=-17.0)})
# Internally, when executed from ANALYST, this is what is going to happen
#
# ```
# conda activate python-training
# python mag_dipole [uijson]
# ```
#
# Then your mag_dipole.run should be able to do the rest: compute and store the result.
#
# Let's write out our ui.json to file.

In [15]:
with open("./test_out.ui.json", "w", encoding="utf-8") as file:
    json.dump(ui_file, file, indent=4)

TypeError: Object of type method is not JSON serializable

 Copyright (c) 2022 Mira Geoscience Ltd.